In [2]:
import requests
import pandas as pd

from global_utilities import get_secret

base_url = "https://skyscanner-skyscanner-flight-search-v1.p.rapidapi.com/apiservices/browsedates/v1.0/"

headers = {
    'x-rapidapi-host': "skyscanner-skyscanner-flight-search-v1.p.rapidapi.com",
    'x-rapidapi-key': get_secret("RAPIDAPI_KEY")
}

In [21]:
from flights.rapidapi import parse_data

from datetime import date, timedelta
from tqdm import tqdm_notebook as tqdm
from time import sleep

In [69]:
dfs = []

In [70]:
def query_flights(origin, destination, day):
    
    for attemp_num in range(10):
        response = requests.get(f"{base_url}ES/EUR/en-US/BCN/CAG/{day:%Y-%m-%d}", headers=headers)
        
        if response.status_code == 200:
            return response
    
        elif response.status_code == 429:
            sleep(0.5 * attemp_num)
            
        else:
            response.raise_for_status()

In [ ]:
for x in tqdm(range(360)):
    day = date.today() + timedelta(x)

    response = query_flights("BCN", "CAG", day)
    
    data = response.json()
    
    if data["Quotes"]:
        dfs.append(parse_data(data))

In [ ]:
df = pd.concat(dfs)

In [ ]:
df